In [7]:
!pip install wordfreq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00


In [1]:
!apt-get install -y imagemagick

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libdjvulibre-text libdjvulibre21
  libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0
  libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10
  libwmflite-0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng enscript gimp
  gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance sane-utils
  texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev inkscape
  poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fon

In [2]:
import os
import random
import numpy as np
import cv2
import pandas as pd
from pathlib import Path
from subprocess import run

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd
df = pd.DataFrame(columns=["code", "word" ,"text_color", "font_size", "font_family", "font_style", "noise"])

In [18]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import random
import io
import os

def to_pil(cv2_img):
    return Image.fromarray(cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB))

def to_cv2(pil_img):
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

def apply_noise(image, noise_type):
    if image is None:
        print(f"Warning: Could not read image. Skipping noise type: {noise_type}")
        return None

    pil_img = to_pil(image)
    cv2_img = image.copy()
    h, w = cv2_img.shape[:2]

    if noise_type == "blur":
        return cv2.GaussianBlur(cv2_img, (5, 5), 0)

    elif noise_type == "motion blur":
        kernel = np.zeros((15, 15))
        kernel[7, :] = np.ones(15)
        kernel /= 15
        return cv2.filter2D(cv2_img, -1, kernel)

    elif noise_type == "pixelation":
        h, w = cv2_img.shape[:2]
        temp = cv2.resize(cv2_img, (w//10, h//10), interpolation=cv2.INTER_LINEAR)
        return cv2.resize(temp, (w, h), interpolation=cv2.INTER_NEAREST)

    elif noise_type == "compression artifacts":
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 10]
        _, enc_img = cv2.imencode('.jpg', cv2_img, encode_param)
        return cv2.imdecode(enc_img, 1)

    elif noise_type == "Gaussian noise":
        row, col, ch = cv2_img.shape
        mean = 0
        sigma = 25
        gauss = np.random.normal(mean, sigma, (row, col, ch))
        gauss = np.clip(gauss, -sigma*3, sigma*3)
        noisy_img = cv2_img.astype(np.float32) + gauss
        noisy_img = np.clip(noisy_img, 0, 255).astype(np.uint8)
        return noisy_img

    elif noise_type == "low resolution":
        small = cv2.resize(cv2_img, (32, 32), interpolation=cv2.INTER_LINEAR)
        return cv2.resize(small, (cv2_img.shape[1], cv2_img.shape[0]), interpolation=cv2.INTER_NEAREST)

    elif noise_type == "image stretching":
        h, w = cv2_img.shape[:2]
        return cv2.resize(cv2_img, (int(w * 1.5), h))

    elif noise_type == "defocus blur":
        return cv2.GaussianBlur(cv2_img, (21, 21), 0)

    elif noise_type == "ringing artifacts":
        blurred = cv2.GaussianBlur(cv2_img, (0, 0), 3)
        return cv2.addWeighted(cv2_img, 1.5, blurred, -0.5, 0)

    elif noise_type == "lighting variations":
        enhancer = ImageEnhance.Brightness(pil_img)
        return to_cv2(enhancer.enhance(random.uniform(0.5, 1.5)))

    elif noise_type == "uneven illumination":
        h, w = cv2_img.shape[:2]
        mask = np.zeros((h, w), np.uint8)
        cv2.circle(mask, (w//2, h//2), w//2, 255, -1)
        mask_3_channel = cv2.merge([mask]*3)
        return cv2.addWeighted(cv2_img, 1, mask_3_channel, 0.3, 0)

    elif noise_type == "shadows":
        mask = np.zeros_like(cv2_img)
        x1 = random.randint(0, w-1)
        y1 = random.randint(0, h-1)
        x2 = random.randint(0, w-1)
        y2 = random.randint(0, h-1)
        cv2.rectangle(mask, (min(x1, x2), min(y1, y2)), (max(x1, x2), max(y1, y2)), (0, 0, 0), -1)
        return cv2.addWeighted(cv2_img, 1, mask, 0.5, 0)

    elif noise_type == "reflections":
        overlay = np.full_like(cv2_img, 255)
        beta_weight = 0.3
        return cv2.addWeighted(cv2_img, 1 - beta_weight, overlay, beta_weight, 0)

    elif noise_type == "glare":
        glare = np.zeros_like(cv2_img)
        center = (random.randint(0, w), random.randint(0, h))
        cv2.circle(glare, center, random.randint(50, 100), (255, 255, 255), -1)
        return cv2.addWeighted(cv2_img, 0.7, glare, 0.3, 0)

    elif noise_type == "overexposure":
        return cv2.convertScaleAbs(cv2_img, alpha=1.5, beta=50)

    elif noise_type == "underexposure":
        return cv2.convertScaleAbs(cv2_img, alpha=0.5, beta=-50)

    elif noise_type == "background clutter":
        noise_bg = np.random.randint(0, 255, cv2_img.shape, dtype='uint8')
        return cv2.addWeighted(cv2_img, 0.8, noise_bg, 0.2, 0)

    elif noise_type == "ambient noise":
        ambient = cv2.GaussianBlur(cv2_img, (9, 9), 10)
        return cv2.addWeighted(cv2_img, 0.7, ambient, 0.3, 0)

    elif noise_type == "perspective distortion":
        pts1 = np.float32([[0,0], [w,0], [0,h], [w,h]])
        pts2 = np.float32([[random.randint(0, w//10), random.randint(0, h//10)],
                           [w-random.randint(0, w//10), random.randint(0, h//10)],
                           [random.randint(0, w//10), h-random.randint(0, h//10)],
                           [w-random.randint(0, w//10), h-random.randint(0, h//10)]])
        M = cv2.getPerspectiveTransform(pts1, pts2)
        return cv2.warpPerspective(cv2_img, M, (w, h))

    elif noise_type == "fading":
        alpha = np.linspace(1, 0.3, w)
        mask = np.tile(alpha, (h, 1))
        return (cv2_img * mask[:, :, np.newaxis]).astype(np.uint8)

    elif noise_type == "ink bleed-through":
        flipped = cv2.flip(cv2_img, 1)
        blended = cv2.addWeighted(cv2_img, 0.8, flipped, 0.2, 0)
        return blended

    elif noise_type == "text smearing":
        kernel = np.ones((1, 9), np.uint8)
        return cv2.dilate(cv2_img, kernel, iterations=1)

    elif noise_type == "stroke breaks":
        mask = np.random.randint(0, 2, size=(h, w, 1), dtype=np.uint8)
        mask_3_channel = cv2.merge([mask[:,:,0]]*3)
        return cv2_img * mask_3_channel

    elif noise_type == "partial occlusion":
        noisy_img = cv2_img.copy()
        for _ in range(3):
            x1, y1 = random.randint(0, max(0, w-50)), random.randint(0, max(0, h-50))
            x2, y2 = x1 + random.randint(20, 50), y1 + random.randint(20, 50)
            cv2.rectangle(noisy_img, (x1, y1), (x2, y2), (0, 0, 0), -1)
        return noisy_img

    elif noise_type == "font erosion":
        kernel = np.ones((2, 2), np.uint8)
        return cv2.erode(cv2_img, kernel, iterations=1)

    elif noise_type == "character merging":
        kernel = np.ones((2, 2), np.uint8)
        return cv2.dilate(cv2_img, kernel, iterations=2)

    elif noise_type == "JPEG compression":
        encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 5]
        _, enc_img = cv2.imencode('.jpg', cv2_img, encode_param)
        decoded_img = cv2.imdecode(enc_img, cv2.IMREAD_COLOR)
        return decoded_img if decoded_img is not None else cv2_img

    elif noise_type == "color space mismatch":
        return cv2.cvtColor(cv2_img, cv2.COLOR_BGR2HSV)

    elif noise_type == "resolution scaling":
        small = cv2.resize(cv2_img, (w//2, h//2), interpolation=cv2.INTER_LINEAR)
        return cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    elif noise_type == "quantization noise":
        levels = 16
        return (cv2_img // levels * levels).astype(np.uint8)

    elif noise_type == "aliasing":
        small = cv2.resize(cv2_img, (w//4, h//4), interpolation=cv2.INTER_LINEAR)
        return cv2.resize(small, (w, h), interpolation=cv2.INTER_NEAREST)

    elif noise_type == "banding":
        banded = cv2_img.copy()
        for i in range(0, h, 20):
            start_row = i
            end_row = min(i + 5, h)
            banded[start_row:end_row] = banded[start_row:end_row] // 2
        return banded

    elif noise_type == "bit-depth reduction":
        return (cv2_img >> 3) << 3

    elif noise_type == "encoding artifacts":
        temp = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(temp, 127, 255, cv2.THRESH_BINARY)
        return cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

    elif noise_type == "scan line artifacts":
        scan_img = cv2_img.copy()
        for i in range(0, h, 5):
             start_row = i
             end_row = min(i + 1, h)
             scan_img[start_row:end_row] = scan_img[start_row:end_row] // 3
        return scan_img

    elif noise_type == "scanner streaks":
        streak = cv2_img.copy()
        for _ in range(5):
            x = random.randint(0, w-1)
            streak[:, x] = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
        return streak

    elif noise_type == "printer toner gaps":
        toner = cv2_img.copy()
        for _ in range(3):
            y = random.randint(0, max(0, h-20)) # Ensure y is valid
            toner[y:min(y+5, h), :] = [255, 255, 255] # White gaps
        return toner

    elif noise_type == "misaligned scanning":
        tx = random.randint(-10, 10)
        ty = random.randint(-5, 5)
        M = np.float32([[1, 0, tx], [0, 1, ty]])
        return cv2.warpAffine(cv2_img, M, (w, h))

    elif noise_type == "fold marks":
        folded = cv2_img.copy()
        for _ in range(2):
            y = random.randint(0, h-1)
            cv2.line(folded, (0, y), (w, y), (200, 200, 200), 2)
        return folded

    elif noise_type == "paper texture interference":
        noise = np.random.normal(127, 25, cv2_img.shape).astype(np.uint8)
        return cv2.addWeighted(cv2_img, 0.9, noise, 0.1, 0)

    elif noise_type == "multi-layer noise":
        noisy = apply_noise(cv2_img, "Gaussian noise")
        if noisy is not None:
          noisy = apply_noise(noisy, "text smearing")
        if noisy is not None:
          return apply_noise(noisy, "JPEG compression")
        return cv2_img

    elif noise_type == "watermarks":
        watermark = cv2.putText(cv2_img.copy(), "WATERMARK", (w//4, h//2),
                                cv2.FONT_HERSHEY_SIMPLEX, min(1.5, h/50), (150, 150, 150), max(1, w//200), cv2.LINE_AA) # Adjust font scale/thickness based on image size
        return cv2.addWeighted(cv2_img, 0.85, watermark, 0.15, 0)

    elif noise_type == "overlay interference":
        overlay = np.full_like(cv2_img, 100)
        return cv2.addWeighted(cv2_img, 0.75, overlay, 0.25, 0)

    else:
        return cv2_img

In [19]:
implemented_noise_types = [
    "blur", "motion blur", "pixelation", "compression artifacts", "Gaussian noise",
    "low resolution", "image stretching", "defocus blur", "ringing artifacts",
    "lighting variations", "uneven illumination", "shadows", "reflections",
    "glare", "overexposure", "underexposure", "background clutter", "ambient noise",
    "perspective distortion", "fading", "ink bleed-through", "text smearing",
    "stroke breaks", "partial occlusion", "font erosion", "character merging",
    "JPEG compression", "color space mismatch", "resolution scaling",
    "quantization noise", "aliasing", "banding", "bit-depth reduction",
    "encoding artifacts", "scan line artifacts", "scanner streaks",
    "printer toner gaps", "misaligned scanning", "fold marks",
    "paper texture interference", "multi-layer noise", "watermarks",
    "overlay interference"
]

In [20]:
import subprocess
import os
import random
from pathlib import Path
import cv2
import numpy as np
from PIL import Image
import shutil


def generate_text_image(
    text: str,
    font_path: str,
    font_size: int,
    output_path: str,
    image_size=(512, 128),
    text_color="black",
    background_color="white"
):
    width, height = image_size
    command = [
        "convert",
        "-size", f"{width}x{height}",
        "xc:" + background_color,
        "-font", str(font_path),
        "-pointsize", str(font_size),
        "-fill", text_color,
        "-gravity", "center",
        "-annotate", "0", text,
        output_path
    ]

    try:
        subprocess.run(command, check=True)
        print(f"Saved clean image: {output_path}")
    except subprocess.CalledProcessError as e:
        print("Failed to generate image:", e)

def generate_and_apply_noise(
    words,
    fonts_dir,
    clean_output_dir,
    noisy_output_dir,
    num_samples=100,
    image_size=(512, 128),
    text_colors=["black", "blue", "red", "green", "purple"],
    background_color="white",
    font_size_range=(32, 128),
    implemented_noise_types=implemented_noise_types
):
    if implemented_noise_types is None:
        implemented_noise_types = [
            "Gaussian noise", "blur", "motion blur", "pixelation"
            # Add your full list here or more implementations in apply_noise()
        ]

    clean_output_dir = Path(clean_output_dir)
    noisy_output_dir = Path(noisy_output_dir)
    clean_output_dir.mkdir(parents=True, exist_ok=True)
    noisy_output_dir.mkdir(parents=True, exist_ok=True)

    font_families = [f for f in Path(fonts_dir).iterdir() if f.is_dir()]
    if not font_families:
        raise ValueError("No font family folders found.")

    image_counter = 0

    for word in words:
        random_family = random.choice(font_families)

        fonts = list(random_family.glob("*.ttf")) + list(random_family.glob("*.otf"))
        if not fonts:
            print(f"No fonts found in {random_family}, skipping...")
            continue

        random_font = random.choice(fonts)
        color = random.choice(text_colors)
        font_name = Path(random_font).stem.replace(" ", "_")
        font_size = random.randint(*font_size_range)

        # File naming for clean image
        clean_filename = f"{image_counter:04d}_{random_family.name}_{font_name}_{color}_{font_size}.png"
        clean_path = clean_output_dir / clean_filename

        # Generate clean synthetic text image
        generate_text_image(
            text=word,
            font_path=random_font,
            font_size=font_size,
            output_path=str(clean_path),
            image_size=image_size,
            text_color=color,
            background_color=background_color
        )

        # Load clean image with OpenCV
        image = cv2.imread(str(clean_path))
        if image is None:
            print(f"Failed to read image {clean_path}, skipping noise application.")
            continue

        # Choose noise type and apply noise
        noise_type = random.choice(implemented_noise_types)
        noisy_image = apply_noise(image, noise_type)

        # Save noisy image with noise type appended in filename
        noisy_filename = f"{clean_path.stem}_{noise_type.replace(' ', '_')}.jpg"
        noisy_path = noisy_output_dir / noisy_filename

        cv2.imwrite(str(noisy_path), noisy_image)
        print(f"Saved noisy image: {noisy_path}")
        df.loc[len(df)] = [
            f"{image_counter:04d}",  # code (id)
            word,                   # word (text)
            color,                  # text_color
            font_size,              # font_size
            random_family.name,       # font_family
            font_name,              # font_style (font file name)
            noise_type              # noise applied
        ]

        image_counter += 1

In [21]:
from wordfreq import top_n_list
all_words = top_n_list('en', 1000)[200:300]
print(len(all_words))
print((all_words))

100
['feel', 'sure', 'team', 'ever', 'family', 'keep', 'might', 'please', 'put', 'money', 'free', 'second', 'someone', 'away', 'left', 'number', 'city', 'days', 'lot', 'name', 'night', 'play', 'until', 'company', 'doing', 'few', "he's", 'let', 'real', '6', 'called', 'different', 'having', 'set', 'thought', 'done', 'however', 'getting', 'god', 'government', 'group', 'looking', 'public', 'top', 'women', 'business', 'care', 'start', 'system', 'times', 'week', '7', 'already', 'anything', 'case', 'nothing', 'person', 'today', 'change', 'enough', 'everything', 'full', 'live', 'making', 'point', 'read', "there's", 'told', 'yet', 'bad', "doesn't", 'four', 'hard', 'mean', 'once', 'support', 'tell', 'including', 'music', 'power', 'seen', 'states', 'stop', 'water', 'based', 'believe', 'call', 'head', 'men', 'national', 'small', 'took', 'white', 'came', 'far', 'job', 'side', 'though', 'try', 'went']


In [22]:
generate_and_apply_noise(
    words=all_words,
    fonts_dir="/content/drive/MyDrive/font-family",  # your fonts folder
    clean_output_dir="/content/drive/MyDrive/synthetic_text_images",  # output clean images
    noisy_output_dir="/content/drive/MyDrive/noisy_images",  # output noisy images
    num_samples=100,
    image_size=(512, 128),
    text_colors=["black", "blue", "red", "green", "purple"],
    font_size_range=(40, 120)
)

Saved clean image: /content/drive/MyDrive/synthetic_text_images/0000_Source_Code_Pro_SourceCodePro-MediumItalic_red_81.png
Saved noisy image: /content/drive/MyDrive/noisy_images/0000_Source_Code_Pro_SourceCodePro-MediumItalic_red_81_defocus_blur.jpg
Saved clean image: /content/drive/MyDrive/synthetic_text_images/0001_Ubuntu_Ubuntu-LightItalic_blue_65.png
Saved noisy image: /content/drive/MyDrive/noisy_images/0001_Ubuntu_Ubuntu-LightItalic_blue_65_motion_blur.jpg
Saved clean image: /content/drive/MyDrive/synthetic_text_images/0002_Great_Vibes_GreatVibes-Regular_purple_40.png
Saved noisy image: /content/drive/MyDrive/noisy_images/0002_Great_Vibes_GreatVibes-Regular_purple_40_glare.jpg
Saved clean image: /content/drive/MyDrive/synthetic_text_images/0003_Nunito_Nunito-Medium_purple_75.png
Saved noisy image: /content/drive/MyDrive/noisy_images/0003_Nunito_Nunito-Medium_purple_75_multi-layer_noise.jpg
Saved clean image: /content/drive/MyDrive/synthetic_text_images/0004_Pacifico_Pacifico-Regu

In [23]:
df

,code,word,text_color,font_size,font_family,font_style,noise
0,0000,feel,red,81,Source_Code_Pro,SourceCodePro-MediumItalic,defocus blur
1,0001,sure,blue,65,Ubuntu,Ubuntu-LightItalic,motion blur
2,0002,team,purple,40,Great_Vibes,GreatVibes-Regular,glare
3,0003,ever,purple,75,Nunito,Nunito-Medium,multi-layer noise
4,0004,family,purple,101,Pacifico,Pacifico-Regular,low resolution
...,...,...,...,...,...,...,...
95,0095,job,green,52,Great_Vibes,GreatVibes-Regular,low resolution
96,0096,side,blue,107,Montserrat_Alternates,MontserratAlternates-Italic,JPEG compression
97,0097,though,red,86,Montserrat_Alternates,MontserratAlternates-ExtraLightItalic,overlay interference
98,0098,try,blue,41,Great_Vibes,GreatVibes-Regular,partial occlusion


In [24]:
df.to_csv('metadata.csv', index=False)